In [ ]:
## BACKUP 2
def generate_random_hermitian_matrix(N, seed, flag):
    np.random.seed(seed)
    
    if flag == 0:
        # Generate using standard normal distribution
        real_part = np.random.randn(N, N)
        imag_part = np.random.randn(N, N)
    elif flag == 1:
        # Generate values between -1 and 1
        real_part = np.random.uniform(-1, 1, (N, N))
        imag_part = np.random.uniform(-1, 1, (N, N))
    else:
        raise ValueError("Invalid flag value. Use 0 for normal distribution or 1 for [-1, 1].")
    
    # Combine real and imaginary parts to form a complex matrix
    A = real_part + 1j * imag_part
    
    # Make the matrix Hermitian
    A = np.tril(A) + np.tril(A, -1).T.conj()  # Lower triangle + conjugate transpose of upper
    
    return A  


def compute_normalized_spacings(N, seed, flag):
    # Generate Hermitian matrix and compute eigenvalues
    A = generate_random_hermitian_matrix(N, seed, flag)
    eigenvalues = np.linalg.eigh(A)[0]  # Sorted real eigenvalues
    
    # Compute spacings and normalize by mean spacing
    spacings = np.diff(eigenvalues)
    avg_spacing = np.mean(spacings)
    normalized_spacings = spacings / avg_spacing
    return A, eigenvalues, normalized_spacings

def calculate_Ps_distribution(N, num_matrices, N_bins, flag):
    all_normalized_spacings = []
    
    # Accumulate normalized spacings from multiple random matrices
    for i in range(num_matrices):
        seed = i  # Use a different seed for each matrix
        A, eigenvalues, normalized_spacings = compute_normalized_spacings(N, seed, flag)
        all_normalized_spacings.extend(normalized_spacings)
    
    # Convert to numpy array
    all_normalized_spacings = np.array(all_normalized_spacings)
    
    # Range of normalized spacings
    DeltaS = np.max(all_normalized_spacings) - np.min(all_normalized_spacings)
    
    # Create bins and count occurrences in each bin
    counts, bin_edges = np.histogram(all_normalized_spacings, bins=N_bins, range=(0, DeltaS))


    # Total number of spacings
    N_tot = len(all_normalized_spacings)
    
    # Compute P(s) for each bin
    P_s = counts / (N_tot)
    
    # Bin centers for plotting
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    return bin_centers, P_s

def plot_distr_and_fit(bin_centers, P_s, num_matrices, N, flag, fitted_P_s):

    if flag == 0:
        msg = "# Generate using standard normal distribution"
    elif flag == 1:
        msg = "# Generate values between -1 and 1"
    else:
        raise ValueError("Invalid flag value. Use 0 for normal distribution, or 1 for [-1, 1].")
    

    # sns.set_theme(style='whitegrid')

    plt.figure(figsize=(10, 6))
    
    plt.plot(bin_centers, P_s, label='$P(s)$', color='blue', linewidth=2)
    plt.plot(bin_centers, fitted_P_s, label='Fitted $P(s)$', color='red', linestyle='--', linewidth=2)

    # Add a title with improved formatting
    plt.title(f'Distribution of Normalized Spacings $P(s)$\nwith {num_matrices} Matrices of Size {N}\n{msg}', 
              fontsize=16)

    # Set labels with larger font sizes
    plt.xlabel('Normalized Spacing $s$', fontsize=14)
    plt.ylabel('$P(s)$', fontsize=14)

    # Add grid for better readability
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)

    # Add a legend with a better location
    plt.legend(fontsize=12, loc='upper right')

    # Set x and y limits if needed for better visibility
    plt.xlim(left=0)  # Adjust this according to your data range
    plt.ylim(bottom=0)  # Adjust this according to your data range

    # Show the plot with a tighter layout
    plt.tight_layout()  # Adjust subplots to fit into figure area.
    plt.show()



def target_function(s, a, b, alpha, beta):
    """Fitting function P(s) = a * (s ** alpha) * exp(b * (s ** beta))"""
    return a * (s ** alpha) * np.exp(b * (s ** beta))

def fitting(bin_centers, P_s, counts, flag):
    # Initial guesses for parameters a, b, alpha, and beta
    initial_guess = [1, -1, 1, 1]
    
    # Perform curve fitting
    params, _ = curve_fit(target_function, bin_centers, P_s, p0=initial_guess, maxfev=5000)
    a, b, alpha, beta = params
    
    # Print fitted parameters

    if flag == 0:
        msg = "# Generate using standard normal distribution"
    elif flag == 1:
        msg = "# Generate values between -1 and 1"
    else:
        raise ValueError("Invalid flag value. Use 0 for normal distribution, or 1 for [-1, 1].")

    print(msg)
    print(f"Fitted parameters:\n a = {a:.4f}, b = {b:.4f}, alpha = {alpha:.4f}, beta = {beta:.4f}")
    
    # Generate fitted values for plotting
    fitted_P_s = target_function(bin_centers, a, b, alpha, beta)

    # Calculate chi-square
    # Approximate uncertainties as sqrt of counts (Poisson uncertainty)
    uncertainties = np.sqrt(counts)
    # Avoid division by zero in case of zero counts
    uncertainties[uncertainties == 0] = 1  # Assign a minimal uncertainty if count is zero
    
    chi_square = np.sum(((P_s - fitted_P_s) ** 2) / (uncertainties ** 2))

    print(f"Chi-square: {chi_square}")
    print('\n')
    
    return a, b, alpha, beta, chi_square, fitted_P_s

## BACKUP 1

import numpy as np
import matplotlib.pyplot as plt

def generate_random_hermitian_matrix(N, seed):
    np.random.seed(seed)
    real_part = np.random.randn(N, N)  # Real part
    imag_part = np.random.randn(N, N)  # Imaginary part
    A = real_part + 1j * imag_part
    A = np.tril(A) + np.tril(A, -1).T.conj()  # Make Hermitian
    return A

def compute_normalized_spacings(N, seed):
    # Generate Hermitian matrix and compute eigenvalues
    A = generate_random_hermitian_matrix(N, seed)
    eigenvalues = np.linalg.eigh(A)[0]  # Sorted real eigenvalues
    
    # Compute spacings and normalize by mean spacing
    spacings = np.diff(eigenvalues)
    avg_spacing = np.mean(spacings)
    normalized_spacings = spacings / avg_spacing
    return A, eigenvalues, normalized_spacings

def calculate_Ps_distribution(N, num_matrices, N_bins):
    all_normalized_spacings = []
    
    # Accumulate normalized spacings from multiple random matrices
    for i in range(num_matrices):
        seed = i  # Use a different seed for each matrix
        A, eigenvalues, normalized_spacings = compute_normalized_spacings(N, seed)
        all_normalized_spacings.extend(normalized_spacings)
    
    # Convert to numpy array
    all_normalized_spacings = np.array(all_normalized_spacings)
    
    # Range of normalized spacings
    DeltaS = np.max(all_normalized_spacings) - np.min(all_normalized_spacings)
    
    # Create bins and count occurrences in each bin
    counts, bin_edges = np.histogram(all_normalized_spacings, bins=N_bins, range=(0, DeltaS))


    # Total number of spacings
    N_tot = len(all_normalized_spacings)
    
    # Compute P(s) for each bin
    P_s = counts / (N_tot)
    print(sum(P_s))
    
    # Bin centers for plotting
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    return bin_centers, P_s

def plot_Ps_distribution(bin_centers, P_s, num_matrices, N):
    plt.figure(figsize=(10, 6))
    plt.plot(bin_centers, P_s, label='$P(s)$')
    plt.title(f'Distribution of Normalized Spacings $P(s)$ with {num_matrices} matrices of size {N}')
    plt.xlabel('Normalized Spacing $s$')
    plt.ylabel('$P(s)$')
    plt.legend()
    plt.grid()
    plt.show()



def target_function(s, a, b, alpha, beta):
    """Fitting function P(s) = a * (s ** alpha) * exp(b * (s ** beta))"""
    return a * (s ** alpha) * np.exp(b * (s ** beta))

def fitting(bin_centers, P_s, counts):
    # Initial guesses for parameters a, b, alpha, and beta
    initial_guess = [1, -1, 1, 1]
    
    # Perform curve fitting
    params, _ = curve_fit(target_function, bin_centers, P_s, p0=initial_guess)
    a, b, alpha, beta = params
    
    # Print fitted parameters
    print(f"Fitted parameters:\n a = {a:.4f}, b = {b:.4f}, alpha = {alpha:.4f}, beta = {beta:.4f}")
    
    # Generate fitted values for plotting
    fitted_P_s = target_function(bin_centers, a, b, alpha, beta)

    # Calculate chi-square
    # Approximate uncertainties as sqrt of counts (Poisson uncertainty)
    uncertainties = np.sqrt(counts)
    # Avoid division by zero in case of zero counts
    uncertainties[uncertainties == 0] = 1  # Assign a minimal uncertainty if count is zero
    
    chi_square = np.sum(((P_s - fitted_P_s) ** 2) / (uncertainties ** 2))
    dof = len(bin_centers) - len(params)  # Degrees of freedom
    chi_square_reduced = chi_square / dof

    print(f"Chi-square: {chi_square:.4f}")
    print(f"Reduced Chi-square: {chi_square_reduced:.4f}")
    
    # Plot the original distribution and the fitted curve
    plt.figure(figsize=(10, 6))
    plt.plot(bin_centers, P_s, label='Original $P(s)$', color='blue')
    plt.plot(bin_centers, fitted_P_s, label=f'Fitted $P(s)$', color='red', linestyle='--')
    plt.title('Fitted Distribution of Normalized Spacings $P(s)$')
    plt.xlabel('Normalized Spacing $s$')
    plt.ylabel('$P(s)$')
    plt.legend()
    plt.grid()
    plt.show()
    
    return a, b, alpha, beta